In [ ]:
!pip install pandas 
!pip install numpy
#!pip install matplotlib
#!pip install datetime
#!pip install seaborn

import pandas as pd 
import numpy as np
#import matplotlib.pyplot as plt
#import datetime as dt
#import seaborn as sns

# Создание вспомогательных функций

Создадим функции, которые упростят процесс вывода всей необходимой информации о датасете.

In [41]:
# функция для обзора датасета
def overview_df(df):
    print(" HEAD OF DATAFRAME ".center(100,'-'), '\n', df.head())
    print(" INFO OF DATAFRAME ".center(100,'-'))
    df.info()
    print(" SHAPE ".center(100,'-'), '\n', 'Rows: {}'.format(df.shape[0]), '\n', 'Columns: {}'.format(df.shape[1]))
    print(" TYPES ".center(100,'-'), '\n', df.dtypes)
    print(" NUMBER OF UNIQUES ".center(100,'-'), '\n', df.nunique())
    print(" MISSING VALUES ".center(100,'-'), '\n', missing_values(df))
    print(" DUPLICATED VALUES ".center(100,'-'), '\n', cheking_duplicates(df))
    print(" DESCRIPTIVE STATISTICS ".center(100,'-'), '\n', df.describe().T)

# функция для поиска пропущенных значений
def missing_values(df):
    mn = df.isnull().sum().sort_values(ascending=False)
    mp = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending=False)
    missing_values = pd.concat([mn, mp], axis=1, keys=['Missing_Number', 'Missing_Percent'])
    return missing_values[missing_values['Missing_Number']>0]

# функция для проверки и подсчета количества дубликатов
def cheking_duplicates(df):
    duplicate_values = df.duplicated(keep='first').sum()
    if duplicate_values > 0:
        df.drop_duplicates(keep='first', inplace=True)
        return 'The number of duplicated values is ' + str(duplicate_values) + '. Duplicates were dropped. New shape is ' + str(df.shape)
    else:
        return 'The number of duplicated values is 0.'

In [ ]:
# Первичный обзор датасета

С помощью написанной функции overview_df(df) выведем ...

In [ ]:
df0 = pd.read_csv(r'C:\Users\Александра\Desktop\customer.csv')
df = df0.copy()

overview_df(df)

# Basic data cleaning

Дубликаты уже были посчитаны и удалены во время составления первичного обзора датасета. 
Далее перейдем к описанию данных, содержащихся в каждом из столбцов, и разберемся, какие из них и почему необходимо почистить.
Почередно проверим, все ли данные из столбцов подходят для анализа, и сделаем выводы о полученной информации.

## Столбец 0 InvoiceDate

In [43]:
# Первая покупка в датасете
print('The minimum date is:', df.InvoiceDate.min())

# Последняя покупка в датасете
print('The maximum date is:', df.InvoiceDate.max())

The minimum date is: 2010-12-01 08:26:00
The maximum date is: 2011-12-09 12:50:00


Из собранных данных можно заметить, что первая запись в датасете относится к 1 декабря 2010 года в 8:26, а последняя — к 9 декабря 2011 года в 12:50.
Исходя из этого, можем исключить декабрь 2011 года из анализа, поскольку нет полной информации об этом месяце.

In [44]:
# Удаление записи из датафрейма за декабрь 2011 года
df = df[df.InvoiceDate < '2011-12-01']

## Столбец 1 InvoiceNo

In [ ]:
# Номера операций/заказов (InvoiceNo), начинающиеся с символа 'C', который указывает на возвращенные или отмененные заказы
df[df['InvoiceNo'].str.startswith('C') == True]

In [46]:
# Удаление всех номеров операций (InvoiceNo), начинающихся с символа 'C'
df = df[df['InvoiceNo'].str.startswith('C') != True]

In [ ]:
# Проверка количества уникальных номеров операций (InvoiceNo)
df.InvoiceNo.nunique()

## Столбец 2 StockCode

In [ ]:
# Проверка количества уникальных биржевых кодов-идентификаторов (StockCode) среди всех записей о продажах
df.StockCode.nunique()

In [ ]:
# Топ-10 биржевых кодов-идентификаторов (StockCode), которые продавались чаще всего
df.StockCode.value_counts().head(10)

## Столбец 3 Description

In [ ]:
# Проверка количества уникальных описаний товаров (Description) среди всех записей о продажах
df.Description.nunique()

In [ ]:
# Топ-10 проданных описаний товаров (Description)
df.Description.value_counts().head(10)

Обратим внимание, что бывают записи, в которых описания товаров (Description) содержат некоторую информацию, не относящуюся к продажам. 

In [ ]:
# Проверка данных, где записи описаний (Description) состоят из '?' или начинаются с '?'
df[df['Description'].str.startswith('?') == True]

In [53]:
# Заметим, что при этом CustomerId = NaN и цена за единицу товара UnitPrice = 0
# Удаление всех полученных записей
df = df[df['Description'].str.startswith('?') != True]

In [ ]:
# Проверяем данных, где записи описаний (Description) начинаются с '*' 
df[df['Description'].str.startswith('*') == True]

In [16]:
# Заметим, что при этом CustomerId = NaN
# Изменение записей на соответствующие описания (Description) без символа '*' и перевод в верхний регистр, как и во всех остальных записях
df['Description'] = df['Description'].replace(('*Boombox Ipod Classic','*USB Office Mirror Ball'),
                                             ('BOOMBOX IPOD CLASSIC','USB OFFICE MIRROR BALL'))

In [ ]:
# 
df[df['Description'].str.islower() == True]['Description'].value_counts()

In [ ]:
# 
df = df[df['Description'].str.islower()!=True]
df.shape

In [ ]:
# 
df[df['Description'].str.istitle()==True]['Description'].value_counts()

In [ ]:
# 
df = df[df['Description'].str.istitle()!=True]
df.shape

In [21]:
#
df['Description'] = df['Description'].str.strip()

## Столбец 4 Quantity

In [ ]:
# 
# 
#df.Quantity.describe() есть в общем обзоре

# 
df[df['Quantity']<0]

In [ ]:
# 
df = df[df['Quantity']>=0]
df.shape

## Столбец 5 UnitPrice

In [ ]:
# 
df.UnitPrice.describe(percentiles=[0.25, 0.5, 0.75, 0.85,0.95, 0.99])

In [25]:
# 
df = df[df['UnitPrice'] != 0.0]

In [ ]:
df.shape

## Столбец 6 CustomerID


In [ ]:
# 
df.CustomerID.nunique()

In [ ]:
# 
df[df.CustomerID.isnull()]

In [29]:
# 
df = df[~df.CustomerID.isnull()]
df.shape

(375312, 11)

## Столбец 7 Age

In [ ]:
df.Age.value_counts(normalize=True)

In [ ]:
# 
df.Age.value_counts().head(10)

## Столбец 8 Gender


In [ ]:
df.Gender.value_counts(normalize=True)

## Столбец 9 Income


In [ ]:
df.Income.value_counts(normalize=True)

In [ ]:
df.Income.value_counts(normalize=True, bins = 10)

## Столбец 10 Country

In [ ]:
# 
df.Country.value_counts(normalize=True)

In [ ]:
# 
df['Country'] = df['Country'].apply(lambda x:'United Kingdom' if x=='United Kingdom' else 'Others')
df.Country.value_counts(normalize=True)

# Вторичный обзор датасета после базовой чистки данных


In [ ]:
# вывод обзора с помощью той же написанной функции
overview_df(df)

# Исследовательский анализ данных (Exploratory Data Analysis - EDA) 


# Когортный анализ


# RFM-анализ
